# Determinando la mejor agrupación de las tiendas de Starbucks según su ubicación geográfica



## 1. Descripción del problema 

Se desea conocer cuál es la mejor forma de agrupar las tiendas de Starbucks, a nivel mundial, según su ubicación geográfica. Con esta información, se puede decidir cómo estructurar los cargos administrativos de la empresa; por ejemplo, algunas compañías toman Centroamérica como una región, mientras que otras tienen operaciones específicas para Costa Rica. 

Para esto, se utilizarán técnicas de aprendizaje no supervisado, específicamente de *clustering*, pues no se conoce con antelación cuál es el mejor agrupamiento de las tiendas. En este contexto, el *mejor agrupamiento* no necesariamente será el óptimo, sino que se refiere al *mejor agrupamiento obtenido por algún algoritmo* pues, como en diversos problemas de aprendizaje no supervisado, no se conoce cuál es la solución óptima al problema.

## 2. Preprocesamiento y limpieza de datos

El preprocesamiento y limpieza de datos se llevó a cabo con **Open Refine**. Se adjunta en el repositorio el historial de acciones realizadas.

### 2.a Eliminar columnas redundantes o que no se utilizarán

Las columnas redundantes son aquellas que poseen poca variabilidad, es decir, que poseen el mismo valor en una gran mayoría de las entradas. En este *dataset*, las columnas **Brand** y **Ownership Type** tienen muy poca variabilidad, pues casi la totalidad de las filas tienen los valores *Starbucks* y *Licensed*, respectivamente. Por esta razón, se eliminan del *dataset*.

Adicionalmente, se eliminan las columnas que no se utilizarán en el trabajo. Específicamente:

- **Store Number**
- **Store Name**
- **Street Address**
- **City**
- **State/Province**
- **Postcode**
- **Phone Number**

### 2.b Cambiar tipo de datos en columnas de Longitud y Latitud

Algunos valores en estas columnas son de tipo entero, mientras que la mayoría son de tipo *double*. Para corregir estas inconsistencias, se convertirán todos los valores en la columna a tipo *double*.

## 3. Transformación de datos

La transformación de datos se llevó a cabo con **Open Refine**. Se adjunta en el repositorio el historial de acciones realizadas.

### 3.a Transformación de la columna Timezone 

En el *dataset* original, esta columna indica la zona horaria, continente y país de ubicación de la tienda. Se aplicó una transformación para que, únicamente, muestre el continente. De esta forma, si el valor original era

> GMT+1:00 Europe/Andorra ,

se transforma a

> Europe.

Esta transformación se llevó a cabo con el siguiente código GREL:

```
# Tiene formato de código
```

### 3.b Creación de columna isUnitedStates a partir de Country

Se incluyó una nueva columna con valores booleanos, llamada *isUnitedStates*, creada a partir de la columna *Country*.

Esta transformación se llevó a cabo con el siguiente código GREL:

```
# Tiene formato de código
```

## 4. Análisis Exploratorio de Datos

### 4.a Estadística Descriptiva

### 4.b Particionamiento en clases

Se utiliza la columna isUnitedStates

## 5. Técnicas de segmentación

Explicar por qué sirven

### 5.a K-Means

### 5.b DBSCAN

## 6. Evaluación de modelos

### 6.a Silhouette Analysis - K-Means

### 6.b Silhouette Analysis - DBSCAN

## 7. Resultados

Gráfica + tabla

## 8. Análisis